In [ ]:
import os
from itertools import product

import numpy as np
import pandas as pd

os.chdir("..")


In [ ]:
# Load data
DATA_DIR = "data"
df_hu_a = pd.read_parquet(os.path.join(DATA_DIR, "human_time_series_a.parquet"))
df_ai_a = pd.read_parquet(os.path.join(DATA_DIR, "ai_time_series_a.parquet"))
df_hu_a = df_hu_a[df_ai_a.columns.to_list()] # reorder columns

df_hu_c = pd.read_parquet(os.path.join(DATA_DIR, "human_time_series_c.parquet"))
df_ai_c = pd.read_parquet(os.path.join(DATA_DIR, "ai_time_series_c.parquet"))
df_ai_c = df_ai_c.loc[:, :, :3] # remove surface contact prediction

In [ ]:
# Define metrics
def mean_absolute_error(y_pred: pd.DataFrame, 
                        y_true: pd.DataFrame):
    ae = (y_pred - y_true).abs()
    mean = ae.mean(axis=0) * 100 # MAE
    error  = ae.std(axis=0) / np.sqrt(len(ae)) * 100
    return mean, error

def mean_absolute_percentage_error(y_pred: pd.DataFrame, 
                                   y_true: pd.DataFrame):
    ape = ((y_pred - y_true) / y_true).abs()
    ape = ape.replace(np.inf, np.nan).fillna(0)
    mean = ape.mean(axis=0) * 100 # MAPE
    error = ape.std(axis=0) / np.sqrt(len(ape)) * 100
    return mean, error

Table 2: Generalization error

In [ ]:
metrics = {"MAE": mean_absolute_error, 
           "MAPE": mean_absolute_percentage_error}
datasets = {"Validation": (df_ai_a, df_hu_a), 
            "Test": (df_ai_c, df_hu_c)}

df = pd.DataFrame(
    index=pd.MultiIndex.from_product([metrics, datasets], 
    names=["Metric", "Dataset"]), 
    columns=df_ai_a.columns)

for metric, dataset in product(metrics.keys(), datasets.keys()):
    mean, std = metrics[metric](*datasets[dataset])
    df.loc[metric, dataset] = list(f"{m:.0f} ± {s:.0f}" for m, s in zip(mean, std))

df